კომპანიის მიერ მოწოდებულია პაციენტების პროფილების ბაზა. ამ ბაზის საშუალებით თქვენი მიზანია მანქანური სწავლების მოდელის შექმნა, რომელიც განახორციელებს დიაგნოზის, კერძოდ კი სიმსივნის, პროგნოზირებას.

თქვენი ამოცანაა დავალებასთან თანდართული მონაცემთა ფაილის ანალიზი, მოდელის შექმნა და დატრენინგებული მოდელის შეფასება.

## კოდების გაშვებისთვის საჭირო ბიბლიოთეკების იმპორტები

In [ ]:
import datetime
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    confusion_matrix,
    f1_score,
    precision_recall_curve,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

## სტილის არჩევა გრაფიკებისთვის

# ფინალური დავალება

In [ ]:
sns.set_style("white")
sns.set_palette("Paired")

## მონაცემთა ანალიზი

ფაილი შედგება 9 სვეტისგან:

* **ასაკი** - პაციენტის ასაკი წლებში
* **ბიოლოგიური გენდერი** - პაციენტის ბიოლოგიური გენდერი. 0 მიუთითებს მამრობით სქესს და 1 მიუთითებს მდედრობითს (0=კაცი; 1=ქალი)
* **BMI** - პაციენტის სხეულის მასის ინდექსი
* **მწეველი** - არის თუ არა პაციენტი მწეველი. 0 მიუთითებს არამწეველს და 1 მიუთითებს მწეველს (0=არამწეველი; 1=მწეველი)
* **გენეტიკური რისკი** - პაციენტის კიბოს გენეტიკური რისკის დონე, 0 მიუთითებს დაბალზე, 1 მიუთითებს საშუალოზე და 2 მიუთითებს მაღალზე (0=დაბალი; 1=საშუალო; 2=მაღალი)
* **ფიზიკური აქტივობა** - პაციენტის მიერ კვირაში ფიზიკურ აქტივობებზე დახარჯული საათების რაოდენობა
* **ალკოჰოლის მიღების დონე** - პაციენტის მიერ კვირაში მოხმარებული ალკოჰოლის ერთეულების რაოდენობა
* **სიმსივნის ისტორია** - აქვს თუ არა პაციენტს სიმსივნის პირადი ისტორია, სადაც 0 მიუთითებს, რომ არ აქვს და 1 მიუთითებს, რომ აქვს (0=არ აქვს; 1=აქვს)
* **დიაგნოზი** - პაციენტის სიმსივნის დიაგნოზის სტატუსი, სადაც 0 მიუთითებს სიმსივნის არარსებობაზე და 1 მიუთითებს სიმსივნეზე (0=ნეგატიური; 1=პოზიტიური)

In [ ]:
df = pd.read_excel("./data/სიმსივნის პრედიქციის მონაცემთა ბაზა.xlsx")
df.head()

ვნახოთ მონაცემების ზომა:

In [ ]:
df.shape

ვნახოთ მონაცემთა ტიპები:

In [ ]:
df.dtypes

ვნახოთ მარტივი აღწერითი სტატისტიკა:

In [ ]:
df.describe()

აუცილებელია დავაკვირდეთ გამოტოვებულ მონაცემებს:

In [ ]:
df.info()

In [ ]:
pd.DataFrame(
    {
        "Number of missing data": df.isna().sum(),
        "Percentage of missing data": (df.isna().sum() / len(df) * 100)
        .round(2)
        .astype(str)
        + "%",
    },
    index=df.columns,
)

გამოტოვებული მონაცემები არ გვაქვს.

სამიზნე ცვლადი არის **დიაგნოზი**, შესაბამისად, დავაკვირდეთ რამდენად დაბალანსირებულია:

In [ ]:
df["დიაგნოზი"].value_counts()

In [ ]:
(df["დიაგნოზი"].value_counts(normalize=True) * 100).round(2).astype(str) + "%"

In [ ]:
def autopct_format(values):
    def my_format(pct):
        total = sum(values)
        val = int(round(pct * total / 100.0))
        return "{:.2f}%\n({v:d})".format(pct, v=val)

    return my_format


plt.figure()

data_to_plot = df["დიაგნოზი"].value_counts().sort_index()
plt.pie(
    data_to_plot,
    labels=["ნეგატიური", "პოზიტიური"],
    autopct=autopct_format(data_to_plot),
    colors=["lightgrey", "lightgreen"],
)

plt.title("დიაგნოზი")
plt.show()

ვნახოთ თითოეული მახასიათებლის განაწილება:

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(14, 16))

sns.histplot(df, x="ასაკი", hue="დიაგნოზი", kde=True, ax=axes[0, 0])
sns.histplot(df, x="BMI", hue="დიაგნოზი", kde=True, ax=axes[0, 1])
sns.histplot(df, x="ფიზიკური აქტივობა", hue="დიაგნოზი", kde=True, ax=axes[1, 0])
sns.histplot(df, x="ალკოჰოლის მიღების დონე", hue="დიაგნოზი", kde=True, ax=axes[1, 1])

sns.countplot(df, x="ბიოლოგიური გენდერი", hue="დიაგნოზი", ax=axes[2, 0])
for container in axes[2, 0].containers:
    axes[2, 0].bar_label(container)

sns.countplot(df, x="მწეველი", hue="დიაგნოზი", ax=axes[2, 1])
for container in axes[2, 1].containers:
    axes[2, 1].bar_label(container)

sns.countplot(df, x="გენეტიკური რისკი", hue="დიაგნოზი", ax=axes[3, 0])
for container in axes[3, 0].containers:
    axes[3, 0].bar_label(container)

sns.countplot(df, x="სიმსივნის ისტორია", hue="დიაგნოზი", ax=axes[3, 1])
for container in axes[3, 1].containers:
    axes[3, 1].bar_label(container)

plt.tight_layout()
plt.show()

დავაკვირდეთ მახასიათებლების კორელაციას:

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(df.corr(), annot=True, square=True, cmap="Blues")
plt.show()

მონაცემების 80% გამოვიყენოთ HPO-სთვის და სატრენინგოდ, ხოლო დარჩენილი 20% სატესტოდ:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["დიაგნოზი"]),
    df["დიაგნოზი"],
    test_size=0.2,
    random_state=21,
    stratify=df["დიაგნოზი"],
)

In [ ]:
print("სატრენინგო მონაცემების ზომა:", X_train.shape, y_train.shape)
print("სატესტო მონაცემების ზომა:", X_test.shape, y_test.shape)

## Logistic Regression vs. Decision Tree vs. Random Forest

შეფასების მეტრიკებად გამოვიყენებთ აკურატულობას, სიზუსტეს, გახსენებას, F1 ქულას, AUROC-ს და AUPRC-ს. საუკეთესო მოდელსა და მის ჰიპერპარამეტრებს ამოვარჩევთ AUROC-ის საშუალებით, რადგანაც **ზედმეტად არადაბალანსირებული სამიზნე ცვლადი არ გვაქვს**.

გავამზადოთ საჭირო ფუნქციები, რომელთაც HPO-სთვის გამოვიყენებთ:

In [ ]:
def get_all_param_combinations(params_dict):
    keys = params_dict.keys()
    values = params_dict.values()

    # ყველა კომბინაციის დაგენერირება
    combinations = [dict(zip(keys, combination)) for combination in product(*values)]

    return combinations


def run_cv(alg, params, X, y, random_state=21):
    accuracies = {}
    precisions = {}
    recalls = {}
    f1s = {}
    aurocs = {}
    auprcs = {}

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)

    for ind, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        X_train, X_valid = X.iloc[train_idx].copy(), X.iloc[valid_idx].copy()
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        # სტანდარტიზაცია სატრენინგო მონაცემების სტატისტიკით
        scaler = StandardScaler()
        scaler.fit(
            X_train[["ასაკი", "BMI", "ფიზიკური აქტივობა", "ალკოჰოლის მიღების დონე"]]
        )
        X_train[["ასაკი", "BMI", "ფიზიკური აქტივობა", "ალკოჰოლის მიღების დონე"]] = (
            scaler.transform(
                X_train[["ასაკი", "BMI", "ფიზიკური აქტივობა", "ალკოჰოლის მიღების დონე"]]
            )
        )
        X_valid[["ასაკი", "BMI", "ფიზიკური აქტივობა", "ალკოჰოლის მიღების დონე"]] = (
            scaler.transform(
                X_valid[["ასაკი", "BMI", "ფიზიკური აქტივობა", "ალკოჰოლის მიღების დონე"]]
            )
        )

        # მოდელის ობიექტის შექმნა
        if alg == "logistic_regression":
            clf = LogisticRegression(random_state=random_state, **params)
        elif alg == "decision_tree":
            clf = DecisionTreeClassifier(random_state=random_state, **params)
        elif alg == "random_forest":
            clf = RandomForestClassifier(random_state=random_state, **params)
        else:
            raise ValueError("Invalid algorithm")

        clf.fit(X_train, y_train)

        # შეფასების მეტრიკების გამოთვლა სატრენინგო და სავალიდაციო მონაცემებზე
        train_preds = clf.predict(X_train)
        valid_preds = clf.predict(X_valid)

        accuracies[f"accuracy_{ind}"] = [
            accuracy_score(y_train, train_preds),
            accuracy_score(y_valid, valid_preds),
        ]
        precisions[f"precision_{ind}"] = [
            precision_score(y_train, train_preds),
            precision_score(y_valid, valid_preds),
        ]
        recalls[f"recall_{ind}"] = [
            recall_score(y_train, train_preds),
            recall_score(y_valid, valid_preds),
        ]
        f1s[f"f1_{ind}"] = [
            f1_score(y_train, train_preds),
            f1_score(y_valid, valid_preds),
        ]
        aurocs[f"auroc_{ind}"] = [
            roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1]),
            roc_auc_score(y_valid, clf.predict_proba(X_valid)[:, 1]),
        ]
        auprcs[f"auprc_{ind}"] = [
            average_precision_score(y_train, clf.predict_proba(X_train)[:, 1]),
            average_precision_score(y_valid, clf.predict_proba(X_valid)[:, 1]),
        ]

    metrics_df = pd.DataFrame(
        dict(
            {"algorithm": [alg, alg]},
            **{
                "params": [
                    params | {"random_state": random_state},
                    params | {"random_state": random_state},
                ]
            },
            **{"set": ["Training", "Validation"]},
            **accuracies,
            **precisions,
            **recalls,
            **f1s,
            **aurocs,
            **auprcs,
        )
    )

    return metrics_df

ჰიპერპარამეტრების სივრცე თითოეული ალგორითმისთვის:

In [ ]:
algorithms_and_params = {
    "logistic_regression": {
        "solver": ["liblinear"],
        "penalty": ["l1", "l2"],
        "C": [0.5, 1, 5, 10],
    },
    "decision_tree": {
        "max_features": ["log2", "sqrt", None],
        "min_samples_leaf": [2, 4],
        "max_depth": np.arange(3, 11),
    },
    "random_forest": {
        "n_estimators": np.arange(10, 60, 10),
        "max_features": ["log2", "sqrt", None],
        "min_samples_leaf": [2, 4],
        "max_depth": np.arange(3, 11),
    },
}

თითოეული კომბინაციით ალგორითმის ტრენინგი და შედეგების შენახვა:

In [ ]:
start_time = datetime.datetime.now()

metrics_dfs = []

for alg, params_dict in algorithms_and_params.items():
    params_list = get_all_param_combinations(params_dict)

    for params in params_list:
        metrics_df = run_cv(alg, params, X_train, y_train)
        metrics_dfs.append(metrics_df)

    print(f"{alg}: HPO is done.")

finish_time = datetime.datetime.now()
print(f"HPO is done in {finish_time - start_time}")

შედეგების გაერთიანება ერთ ცხრილად:

In [ ]:
master_metrics_df = pd.concat(metrics_dfs, ignore_index=True)

In [ ]:
len(master_metrics_df)

ჯამში ყველა მოდელისთვის გვქონდა ჰიპერპარამეტრების 296 კომბინაცია. რადგანაც როგორც სატრენინგო, ასევე სავალიდაციო ნაწილების შედეგებიც შევინახეთ, ამიტომ ცხრილში გვაქვს 592 ჩანაწერი.

თითოეული შეფასების მეტრიკისთვის სატრენინგო და სავალიდაციო ნაწილების საშუალო არითმეტიკულისა და სტანდარტული გადახრის გამოთვლა:

In [ ]:
master_metrics_df["accuracy_avg"] = master_metrics_df[
    ["accuracy_0", "accuracy_1", "accuracy_2"]
].mean(axis="columns")
master_metrics_df["accuracy_std"] = master_metrics_df[
    ["accuracy_0", "accuracy_1", "accuracy_2"]
].std(axis="columns")

master_metrics_df["precision_avg"] = master_metrics_df[
    ["precision_0", "precision_1", "precision_2"]
].mean(axis="columns")
master_metrics_df["precision_std"] = master_metrics_df[
    ["precision_0", "precision_1", "precision_2"]
].std(axis="columns")

master_metrics_df["recall_avg"] = master_metrics_df[
    ["recall_0", "recall_1", "recall_2"]
].mean(axis="columns")
master_metrics_df["recall_std"] = master_metrics_df[
    ["recall_0", "recall_1", "recall_2"]
].std(axis="columns")

master_metrics_df["f1_avg"] = master_metrics_df[["f1_0", "f1_1", "f1_2"]].mean(
    axis="columns"
)
master_metrics_df["f1_std"] = master_metrics_df[["f1_0", "f1_1", "f1_2"]].std(
    axis="columns"
)

master_metrics_df["auroc_avg"] = master_metrics_df[
    ["auroc_0", "auroc_1", "auroc_2"]
].mean(axis="columns")
master_metrics_df["auroc_std"] = master_metrics_df[
    ["auroc_0", "auroc_1", "auroc_2"]
].std(axis="columns")

master_metrics_df["auprc_avg"] = master_metrics_df[
    ["auprc_0", "auprc_1", "auprc_2"]
].mean(axis="columns")
master_metrics_df["auprc_std"] = master_metrics_df[
    ["auprc_0", "auprc_1", "auprc_2"]
].std(axis="columns")

საშუალო AUROC-ზე დაყრდნობით სავალიდაციო მონაცემებში საუკეთესო ალგორითმისა და მისი ჰიპერპარამეტრების ამორჩევა:

In [ ]:
best_params = master_metrics_df.loc[
    master_metrics_df[master_metrics_df["set"] == "Validation"]["auroc_avg"].argmax()
]["params"]

In [ ]:
master_metrics_df[master_metrics_df["params"] == best_params]

In [ ]:
best_params

როგორც შედეგებიდან ჩანს, საუკეთესო მოდელია Random Forest.

## საბოლოო მოდელის ტრენინგი და შეფასება

რა თქმა უნდა, მონაცემები მსგავსად უნდა დამუშავდეს, როგორც ჯვარედინი ვალიდაციისას მოხდა:

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train[["ასაკი", "BMI", "ფიზიკური აქტივობა", "ალკოჰოლის მიღების დონე"]])
X_train[["ასაკი", "BMI", "ფიზიკური აქტივობა", "ალკოჰოლის მიღების დონე"]] = (
    scaler.transform(
        X_train[["ასაკი", "BMI", "ფიზიკური აქტივობა", "ალკოჰოლის მიღების დონე"]]
    )
)
X_test[["ასაკი", "BMI", "ფიზიკური აქტივობა", "ალკოჰოლის მიღების დონე"]] = (
    scaler.transform(
        X_test[["ასაკი", "BMI", "ფიზიკური აქტივობა", "ალკოჰოლის მიღების დონე"]]
    )
)

დავატრენინგოთ Random Forest საუკეთესო ჰიპერპარამეტრებით:

In [ ]:
rf = RandomForestClassifier(**best_params)
rf.fit(X_train, y_train)

ვნახოთ მისი შეფასების მეტრიკები როგორც სატრენინგო, ასევე სატესტო მონაცემებზე:

In [ ]:
train_preds = rf.predict(X_train)
test_preds = rf.predict(X_test)

metrics_df = pd.DataFrame(
    {
        "accuracy": [
            accuracy_score(y_train, train_preds),
            accuracy_score(y_test, test_preds),
        ],
        "precision": [
            precision_score(y_train, train_preds),
            precision_score(y_test, test_preds),
        ],
        "recall": [
            recall_score(y_train, train_preds),
            recall_score(y_test, test_preds),
        ],
        "f1": [f1_score(y_train, train_preds), f1_score(y_test, test_preds)],
        "auroc": [
            roc_auc_score(y_train, rf.predict_proba(X_train)[:, 1]),
            roc_auc_score(y_test, rf.predict_proba(X_test)[:, 1]),
        ],
        "auprc": [
            average_precision_score(y_train, rf.predict_proba(X_train)[:, 1]),
            average_precision_score(y_test, rf.predict_proba(X_test)[:, 1]),
        ],
    },
    index=["Training", "Test"],
)
metrics_df

დავაკვირდეთ დაბნეულობის მატრიცას სატრენინგო და სატესტო მონაცემებზე:

In [ ]:
fig, (ax_1, ax_2) = plt.subplots(1, 2, figsize=(14, 7))

train_cm = confusion_matrix(y_train, train_preds)
sns.heatmap(train_cm, annot=True, cmap="Blues", fmt="g", square=True, ax=ax_1)
ax_1.tick_params(axis="y", labelrotation=0)
ax_1.set_title("Training")
ax_1.set_ylabel("Actual Label")
ax_1.set_xlabel("Predicted Label")


test_cm = confusion_matrix(y_test, test_preds)
sns.heatmap(test_cm, annot=True, cmap="Blues", fmt="g", square=True, ax=ax_2)
ax_2.tick_params(axis="y", labelrotation=0)
ax_2.set_title("Test")
ax_2.set_ylabel("Actual Label")
ax_2.set_xlabel("Predicted Label")

plt.show()

დავაკვირდეთ ROC მრუდს სატრენინგო და სატესტო მონაცემებზე:

In [ ]:
plt.figure(figsize=(10, 6))

y_train_pred_proba = rf.predict_proba(X_train)[:, 1]
train_fpr, train_tpr, _ = roc_curve(y_train, y_train_pred_proba)
train_auc = roc_auc_score(y_train, y_train_pred_proba)
plt.plot(train_fpr, train_tpr, label=f"Training (AUC={train_auc:.2f})")

y_test_pred_proba = rf.predict_proba(X_test)[:, 1]
test_fpr, test_tpr, _ = roc_curve(y_test, y_test_pred_proba)
test_auc = roc_auc_score(y_test, y_test_pred_proba)
plt.plot(test_fpr, test_tpr, label=f"Test (AUC={test_auc:.2f})")

plt.plot([0, 1], [0, 1], linestyle="--", color="grey", label="Random (AUC=0.5)")

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.show()

დავაკვირდეთ PR მრუდს სატრენინგო და სატესტო მონაცემებზე:

In [ ]:
plt.figure(figsize=(10, 6))

y_train_pred_proba = rf.predict_proba(X_train)[:, 1]
train_precision, train_recall, _ = precision_recall_curve(y_train, y_train_pred_proba)
train_auc = average_precision_score(y_train, y_train_pred_proba)
plt.plot(train_recall, train_precision, label=f"Training (AUC={train_auc:.2f})")

y_test_pred_proba = rf.predict_proba(X_test)[:, 1]
test_precision, test_recall, _ = precision_recall_curve(y_test, y_test_pred_proba)
test_auc = average_precision_score(y_test, y_test_pred_proba)
plt.plot(test_recall, test_precision, label=f"Test (AUC={test_auc:.2f})")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall (PR) Curve")
plt.legend(loc="lower left")
plt.show()